<a href="https://colab.research.google.com/github/zualexander/mai-aiapp-computer-vision/blob/main/mai_computer_vision_experiment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tarfile
import os
import splitfolders
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

#Functions

In [3]:
#https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
def plot_accuracy_and_loss(history):

  # summarize history for accuracy
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('model accuracy')
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()
  # summarize history for loss
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

# Download Dataset
from [howto](https://towardsdatascience.com/an-informative-colab-guide-to-load-image-datasets-from-github-kaggle-and-local-machine-75cae89ffa1e)

In [4]:
!wget --no-check-certificate \
    ""http://aisdatasets.informatik.uni-freiburg.de/freiburg_groceries_dataset/freiburg_groceries_dataset.tar.gz"" \
    -O "/tmp/dataset.tar.gz"

--2022-05-26 22:43:43--  http://aisdatasets.informatik.uni-freiburg.de/freiburg_groceries_dataset/freiburg_groceries_dataset.tar.gz
Resolving aisdatasets.informatik.uni-freiburg.de (aisdatasets.informatik.uni-freiburg.de)... 132.230.105.132
Connecting to aisdatasets.informatik.uni-freiburg.de (aisdatasets.informatik.uni-freiburg.de)|132.230.105.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 541562880 (516M) [application/x-gzip]
Saving to: ‘/tmp/dataset.tar.gz’

/tmp/dataset.tar.gz 100%[===================>] 516.47M  20.1MB/s    in 26s     

2022-05-26 22:44:09 (19.8 MB/s) - ‘/tmp/dataset.tar.gz’ saved [541562880/541562880]



In [5]:
!mkdir /tmp/dataset 
!tar xf /tmp/dataset.tar.gz --directory=/tmp/dataset

# Datasplit

In [6]:
splitfolders.ratio("/tmp/dataset/images", output="/tmp/dataset/output",
    seed=1337, ratio=(.75, .25), group_prefix=None, move=False)

Copying files: 4947 files [00:01, 2498.24 files/s]


In [7]:
ls /tmp/dataset/output/train

BEANS/   CHIPS/      FISH/   JUICE/  PASTA/   SUGAR/         WATER/
CAKE/    CHOCOLATE/  FLOUR/  MILK/   RICE/    TEA/
CANDY/   COFFEE/     HONEY/  NUTS/   SODA/    TOMATO_SAUCE/
CEREAL/  CORN/       JAM/    OIL/    SPICES/  VINEGAR/


In [8]:
train = keras.utils.image_dataset_from_directory(
    directory='/tmp/dataset/output/train/',
    labels='inferred',
    label_mode='categorical',
    crop_to_aspect_ratio=True,
    batch_size=32,
    image_size=(256, 256))

validation = keras.utils.image_dataset_from_directory(
    directory='/tmp/dataset/output/val/',
    labels='inferred',
    label_mode='categorical',
    crop_to_aspect_ratio=True,
    batch_size=32,
    image_size=(256, 256))


Found 3699 files belonging to 25 classes.
Found 1248 files belonging to 25 classes.


In [9]:
print(len(train.class_names))

25


# Model

In [ ]:
resnet_model = keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(256, 256, 3),
    pooling=None,
    classes=len(train.class_names)
) 

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
resnet_model.summary()

In [ ]:
resnet_model.count_params()

In [ ]:
model = keras.models.Sequential()

for layer in resnet_model.layers:
        layer.trainable=False

model.add(resnet_model)

In [ ]:
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(len(train.class_names), activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.count_params()

In [ ]:
#compile model
model.compile(
  optimizer=keras.optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=['acc'])

In [ ]:
keras.utils.plot_model(model,show_shapes=True)

In [ ]:
hist = model.fit(train, validation_data=validation, epochs=50)

#Model evaluation

##trainigs history

In [ ]:
print ("test accuracy ", hist.history['acc'][-1])

#validation history

In [ ]:
print ("validation accuracy ", hist.history['val_acc'][-1])

#accuracy and loss plot

In [ ]:
print(hist.history.keys())
plot_accuracy_and_loss(hist)

#Confusion matrix

In [ ]:
y_predict = model.predict(valdiation)